In [1]:
## this class will load index etc, and provide results...
import utils
import torch
from inverted_index import InvertedIndex
from datasets import load_dataset
import spacy
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import \
    AutoModel, AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline, pipeline
from retrieval import Retrieval
import retrieval

In [2]:
import importlib

# Reload the module
importlib.reload(utils)
importlib.reload(retrieval)

<module 'retrieval' from '/disks/disk00/enrico_benedetti/nihongoexample/code/scripts/retrieval.py'>

In [3]:
config = {'dataset' : "bennexx/jp_sentences",
          'spacy_model': 'ja_ginza',
          'index_file': '/data/enrico_benedetti/nihongoexample/data/corpus_all/jp_sentences/inverted_index.pkl',
          'sense_model': 'bennexx/mirrorwic-cl-tohoku-bert-base-japanese-v3',
          #'diff_model': "/data/enrico_benedetti/nihongoexample/code/models/checkpoint-66"
          'diff_model': 'bennexx/cl-tohoku-bert-base-japanese-v3-jlpt-classifier',
          # 'device': 'cpu',
          'device': 0,
          'save_dir' : '../../evaluation/outputs/experiments',
          #'generation': True
          }

In [ ]:
# config['generation']

In [ ]:
# index = retr.index
# nlp = retr.nlp
# df = retr.df

In [4]:
retr = Retrieval(config)

/home/enrico_benedetti/anaconda3/envs/nlp_env/lib/python3.8/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'ja_ginza' (5.1.3) was trained with spaCy v3.2.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Index loaded: /disks/disk00/enrico_benedetti/nihongoexample/data/corpus_all/jp_sentences/inverted_index.pkl


In [16]:
# l, _ = retr.get_sentence_list('日本語能力試験', '日本語能力試験は簡単です。', target_level='N5')
# l

In [18]:
candidates

,sentence,level,level_score
0,日本語能力試験は簡単です。,N5,1.0
1,日本語能力試験は簡単です。,N5,1.0
2,日本語能力試験は簡単です。,N5,1.0
3,日本語能力試験は簡単です。,N5,1.0
4,日本語能力試験は簡単です。,N5,1.0


In [21]:
## generation test

candidates = ['自分の意思で、内部に入ろうと思わなければ、中には入れない。','私は部屋に入る前に、靴を脱ぎます。', '犬が庭に入ると、鳥が飛び立ちます。', '学校に入る前に、朝ごはんを食べるのが大切です。', 'バスに乗る前に、切符を買ってください。', '自分の家に入ると、いつもリラックスできます。']
#candidates = ['日本語能力試験は簡単です。']*6
candidates = pd.DataFrame({'sentence': candidates})
candidates
final_list, _ = retr.get_sentence_list('入る', '自分の意思で、内部に入ろうと思わなければ、中には入れない。', candidates=candidates, target_level='N5')
final_list

/home/enrico_benedetti/anaconda3/envs/nlp_env/lib/python3.8/site-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


,sentence,level,level_score,sense_score,quality_score,sentence_docs,parse_tree,tokenized,syntax_div_score,lexical_div_score,div_score,total_score
0,自分の意思で、内部に入ろうと思わなければ、中には入れない。,N3,0.2,0.000000,0.100000,"(自分, の, 意思, で, 、, 内部, に, 入ろう, と, 思わ, なけれ, ば, 、...",[[[(NOUN_advcl (NOUN_nmod ADP_case) AUX_cop PU...,"[自分, の, 意思, で, 、, 内部, に, 入ろう, と, 思わ, なけれ, ば, 、...",NaN,NaN,NaN,NaN
1,私は部屋に入る前に、靴を脱ぎます。,N5,1.0,0.709723,0.854861,"(私, は, 部屋, に, 入る, 前, に, 、, 靴, を, 脱ぎ, ます, 。)","[[ADP_case], [[(NOUN_obl ADP_case)], ADP_case,...","[私, は, 部屋, に, 入る, 前, に, 、, 靴, を, 脱ぎ, ます, 。]",0.393987,0.945312,0.669650,0.762256
2,犬が庭に入ると、鳥が飛び立ちます。,N4,0.6,0.663189,0.631595,"(犬, が, 庭, に, 入る, と, 、, 鳥, が, 飛び立ち, ます, 。)","[[[ADP_case], [ADP_case], SCONJ_mark, PUNCT_pu...","[犬, が, 庭, に, 入る, と, 、, 鳥, が, 飛び立ち, ます, 。]",0.440480,0.908259,0.674370,0.652982
3,自分の家に入ると、いつもリラックスできます。,N4,0.6,0.629022,0.614511,"(自分, の, 家, に, 入る, と, 、, いつも, リラックス, でき, ます, 。)","[[[(NOUN_nmod ADP_case), ADP_case], SCONJ_mark...","[自分, の, 家, に, 入る, と, 、, いつも, リラックス, でき, ます, 。]",0.424218,0.852033,0.638126,0.626318
4,学校に入る前に、朝ごはんを食べるのが大切です。,N4,0.6,0.519222,0.559611,"(学校, に, 入る, 前, に, 、, 朝ごはん, を, 食べる, の, が, 大切, で...","[[[(VERB_acl (NOUN_obl ADP_case)), ADP_case, P...","[学校, に, 入る, 前, に, 、, 朝ごはん, を, 食べる, の, が, 大切, で...",0.429024,0.812508,0.620766,0.590189
5,バスに乗る前に、切符を買ってください。,N5,1.0,-1.000000,0.000000,"(バス, に, 乗る, 前, に, 、, 切符, を, 買っ, て, ください, 。)","[[[(NOUN_obl ADP_case)], ADP_case, PUNCT_punct...","[バス, に, 乗る, 前, に, 、, 切符, を, 買っ, て, ください, 。]",0.475755,0.811285,0.643520,0.321760


In [ ]:
sentence_list, short_list = retr.get_sentence_list('日本語能力試験', '日本語能力試験は簡単です。', target_level='N5')
#sentence_list, short_list = retr.get_sentence_list('良い', '味噌を食事の一部にすることは非常に良いと考えられている。', target_level='N5')
sentence_list

In [ ]:
# target_df = pd.read_csv('../../data/targets/target_words.csv')
# levels = ['N1', 'N2', 'N3', 'N4', 'N5']
# for level in levels:
#     for id, row in tqdm(target_df.iterrows()):
#         print(f'level {level}, searching for {row["target_word"]}...')
#         target_word = row['target_word']
#         context_sentence = row['context_sentence']
#         list_ids, _, _ = retr.get_candidates(target_word, context_sentence)
#         target_df.loc[id, 'number_hits_in_corpus_cap'] = len(list_ids) - 1 # because the context sentence is here
#         list_ids, _, _ = retr.get_candidates(target_word, context_sentence, max_hits=100000000)
#         target_df.loc[id, 'number_hits_in_corpus_tot'] = len(list_ids) - 1 # because the context sentence is here

# target_df['number_hits_in_corpus_cap'] = target_df['number_hits_in_corpus_cap'].astype(int)
# target_df['number_hits_in_corpus_tot'] = target_df['number_hits_in_corpus_tot'].astype(int)
# # target_df.drop('number_hits_in_corpus', axis=1, inplace=True)
# target_df.to_csv('../../data/targets/target_words.csv', index=False)

In [ ]:
## 
# read the file of target words
# search for every level, every word + context

# target_df = pd.read_csv('../../data/targets/target_words.csv')
# levels = ['N1', 'N2', 'N3', 'N4', 'N5']
# for level in levels:
#     for id, row in tqdm(target_df.iterrows()):
#         print(f'level {level}, searching for {row["target_word"]}...')
#         target_word = row['target_word']
#         context_sentence = row['context_sentence']
#         try:
#             retr.get_sentence_list(target_word, context_sentence, k=5, target_level=level)
#         except Exception as e:
#             print(e)
#     break

In [ ]:
import os
target_df = pd.read_csv('../../data/targets/target_words.csv')
# Define the directory where your CSV files are located
directory = '/data/enrico_benedetti/nihongoexample/evaluation/outputs/retrieval/'

# Create a dictionary to store the counts for each file
file_counts = {}

# List all files in the directory
file_list = os.listdir(directory)

# Iterate through the list of files
for filename in file_list:
    # Split the filename into parts using '_' as a delimiter
    parts = filename.split('_')
    
    # Extract the file name without the version information
    if len(parts) < 3:
        continue
    base_filename = parts[-3]
    #print(base_filename)
    
    # If the base filename is already in the dictionary, increment the count
    if base_filename in file_counts:
        file_counts[base_filename] += 1
    else:
        # If it's not in the dictionary, add it with a count of 1
        file_counts[base_filename] = 1

missing = []
# Iterate through the dictionary and print files with less than 5 versions
for filename, count in file_counts.items():
    if count < 5:
        print(f"File '{filename}' has {count} versions (less than 5).")
        missing.append(filename)


In [ ]:
# fix manually the missing versions
# target_word = 'やる'
# mask = target_df['target_word'] == target_word
# context = target_df.loc[mask,'context_sentence'].values[0]
# context
# sentence_list, short_list = retr.get_sentence_list(target_word, context, target_level='N4')
# sentence_list, short_list = retr.get_sentence_list(target_word, context, target_level='N5')